# Proyecto de Transparencia - Webscraping

In [17]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
def get_sections(table_rows):
    """
    Sacar datos, columnas y titulos de todas las tablas
    """
    datos_azul = []
    criterios_persona = []
    criterios = []
    titulos = []
    for table_row in table_rows:
        data = table_row.find_all('td', {'class': 'tddatosazul'})
        criterio_persona = table_row.find_all('td', {'class': 'tdcriterioPer'})
        criterio = table_row.find_all('td', {'class': 'tdcriterio'})
        titles = table_row.find_all('td', {'class': 'simpletextmayor'})

        datos_azul.append(data)
        criterios_persona.append(criterio_persona)
        criterios.append(criterio)
        titulos.append(titles)
    return datos_azul, criterios_persona, criterios, titulos

In [3]:
def get_perfil_legislador(datos_azul, criterios_persona):
    """
    Obtener tabla 'perfil del legislador'
    Su estructura es diferente al resto de las demas tablas
    """
    datos_perfil_legislador = datos_azul[0:puntos_corte[0]]
    atributos_perfil_legislador = criterios_persona[0:puntos_corte[0]]

    atributos_perfil_legislador[3]
    datos_perfil_legislador[3]

    dato_legislador = []
    atributo_legislador = []
    for dato, atributo in zip(datos_perfil_legislador[3], atributos_perfil_legislador[3]):
        dato_legislador.append(dato.text.strip())
        atributo_legislador.append(atributo.text.strip())
    df = pd.DataFrame([])
    df['Atributo'] = atributo_legislador
    df['Datos'] =dato_legislador
    return df

In [4]:
def get_tables_locations(titulos):
    """
    Obtener ubicaciones de todas las tablas en el html
    """
    i = 0
    puntos_corte = []
    while i < len(titulos):
        if (len(titulos[i]) > 0):
            puntos_corte.append(i)
            i += 1
        else:
            i += 1
    return puntos_corte    

In [5]:
def get_criterios(datos_azul, criterios, num_tabla):
    """
    Sacar datos, nombre de columnas y titulos de la tabla "i"
    """
    try:
        table_title = titulos[puntos_corte[num_tabla]][0].text.strip()
        data = datos_azul[puntos_corte[num_tabla]:puntos_corte[num_tabla + 1]]
        criteria = criterios[puntos_corte[num_tabla]:puntos_corte[num_tabla + 1]]
    except:
        table_title = titulos[puntos_corte[num_tabla]][0].text.strip()
        data = datos_azul[puntos_corte[num_tabla]:]
        criteria = criterios[puntos_corte[num_tabla]:]

    for criterion in criteria:
        if len(criterion) > 0:
            table_criteria = criterion
    return data, table_criteria, table_title

In [6]:
def get_table(table_criteria, table_data):
    """
    Parsear la tabla "i" a un dataframe
    """
    lista_dics = []
    for data in table_data:
        if len(data) > 0:
            num_elementos = len(table_criteria)
            i = 0
            d = {}
            while (i < num_elementos):
                d[table_criteria[i].text.strip()] = data[i].text.strip()
                i += 1
            lista_dics.append(d)
    df = pd.DataFrame.from_dict(lista_dics)
    return df

# Pipeline

- Vamos a sacar todas las tablas de golpe
- Ya sabemos cuántas son y tenemos las funciones necesarias

## Sacar HTML con BeautifulSoup

In [7]:
url_1 = "http://sil.gobernacion.gob.mx/Librerias/pp_PerfilLegislador.php?SID=&Referencia=9221837"
url_2 = "http://sil.gobernacion.gob.mx/Librerias/pp_PerfilLegislador.php?SID=&Referencia=9221831"
url_3 = "http://sil.gobernacion.gob.mx/Librerias/pp_PerfilLegislador.php?SID=&Referencia=9221843"

In [68]:
response = requests.get(url_1)
html_doc = response.content
soup = BeautifulSoup(html_doc, 'html.parser')

### Vamos a jalar los tr's y de ahí sacaremos los datos que sean necesarios
- Los _table rows_ (tr) traen toda la info que necesitamos (pero desordenada)


In [69]:
table_rows = soup.find_all('tr')

### Obtenemos secciones con la info. que queremos (datos, criterios de la primera tabla, nombres de las columnas, etc)

In [70]:
datos_azul, criterios_persona, criterios, titulos = get_sections(table_rows)

### Sacar primera tabla (perfil legislador)

### Las partes donde hay/no hay subtitulos nos van a decir las tablas faltantes por sacar y en dónde están

In [71]:
puntos_corte = get_tables_locations(titulos)
print("Ubicaciones tablas: ", puntos_corte)
print("Numero de tablas: ", len(puntos_corte))

Ubicaciones tablas:  [20, 29, 32, 42, 50]
Numero de tablas:  5


In [72]:
perfil_legislador = get_perfil_legislador(datos_azul, criterios_persona)
perfil_legislador

,Atributo,Datos
0,Nombre:,"Senador Propietario:Gutiérrez Castorena, Danie..."
1,Estatus:,Activo
2,Periodo de la legislatura:,Del 29/08/2018 al 31/08/2021
3,Partido:,Morena
4,Nacimiento:,Fecha: N/AEntidad: Ciudad:
5,Principio de elección:,Primera Minoría
6,Zona:,Entidad: Aguascalientes
7,Toma de protesta:,29/08/2018
8,Ubicación en la cámara:,"Edif., del Hemiciclo Piso 4\r\nOficina 16, Col..."
9,Correo electrónico:,oficina.daniel.gutierrezc@senado.gob.mx


In [73]:
numeros_tablas = np.arange(0, len(puntos_corte))
numeros_tablas

array([0, 1, 2, 3, 4])

### Obtener datos de la tabla "i" junto con el nombre de sus columnas y parsearlas a un dataframe

In [74]:
lista_tablas = []
for numero_tabla in numeros_tablas:
    table_data, table_criteria, table_title = get_criterios(datos_azul, criterios, numero_tabla)
    print(table_title)
    df_table = get_table(table_criteria, table_data)
    lista_tablas.append(df_table)

COMISIONES
TRAYECTORIA ADMINISTRATIVA
TRAYECTORIA POLÍTICA
TRAYECTORIA ACADÉMICA
OTROS RUBROS


In [76]:
lista_tablas[3]

,Del año,Al año,Trayectoria
0,1974,1978,Licenciatura en Ciencias Políticas y Administr...
1,1978,1984,Docente de la Universidad Autónoma de Aguascal...
2,1985,2015,Docente investigador de la Universidad Autónom...
3,1992,1993,Maestría en Ciencias Sociales por la Universid...
4,1994,1996,Maestría en Sociología Industrial y del Trabaj...
5,1998,2001,Doctorado en Estudios Sociales por la UAM.
